In [ ]:
# Basic imports
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import subprocess
from matplotlib import pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import ConcatDataset, DataLoader
import os

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.dropout1 = nn.Dropout2d(0.25)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.dropout2 = nn.Dropout2d(0.25)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.batch_norm3 = nn.BatchNorm2d(128)
        self.dropout3 = nn.Dropout2d(0.25)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(128 * 3 * 3, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.batch_norm1(self.conv1(x)))
        x = self.dropout1(x)
        x = self.pool1(x)

        x = F.relu(self.batch_norm2(self.conv2(x)))
        x = self.dropout2(x)
        x = self.pool2(x)

        x = F.relu(self.batch_norm3(self.conv3(x)))
        x = self.dropout3(x)
        x = self.pool3(x)

        x = x.view(-1, 128 * 3 * 3)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = CNNModel()

In [ ]:
def to_binary(image):
    threshold = 0.5
    return (image > 0).float()

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Lambda(to_binary)
])

training_data_mnist = torchvision.datasets.MNIST(root="../data", train=True, transform=transform, download=True)
test_data_mnist = torchvision.datasets.MNIST(root="../data", train=False, transform=transform, download=True)

training_data_emnist = torchvision.datasets.EMNIST(root="../data", split="digits", train=True, transform=transform, download=True)
test_data_emnist = torchvision.datasets.EMNIST(root="../data", split="digits", train=False, transform=transform, download=True)

full_dataset_train = ConcatDataset([training_data_mnist, training_data_emnist])
full_dataset_test = ConcatDataset([test_data_mnist, test_data_emnist])

batch_size = 64
data_loader_train = DataLoader(full_dataset_train, batch_size=batch_size, shuffle=True)
data_loader_test = DataLoader(full_dataset_test, batch_size=batch_size, shuffle=True)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [ ]:
Epochs = 3
for epoch in range(Epochs):
  training_loss = 0.0
  correct = 0
  total = 0
  for i, data in enumerate(data_loader_train, 0):
    inputs, labels = data
    optimizer.zero_grad()

    outputs = model(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    training_loss += loss.item()
    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()
    avg_loss = training_loss / (i + 1)
    avg_acc = 100. * correct / total
  print(f'Training Loss: {avg_loss:.3f} | Training acc: {avg_acc:.3f}', 'for epoch: ', epoch)

In [ ]:
torch.save(model.state_dict(), 'cnn.pth')
print("Model saved to 'cnn.pth'")

In [ ]:
import time
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for i, (image, label) in enumerate(data_loader_test):
      output = model(image)
      test_loss += F.nll_loss(output, label, reduction='sum').item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(label.data.view_as(pred)).sum()
      plt.imshow(image[0].squeeze(0), cmap="gray")
      plt.title(f"Prediction: {pred[0].item()}, Label: {label[0].item()}")
      plt.axis("off")
      plt.show()
      time.sleep(3)
    test_loss /= len(data_loader_test.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(data_loader_test.dataset), 100. * correct / len(data_loader_test.dataset)))
